In [18]:
import argparse
import cv2 as cv
import numpy as np
from numpy.fft import fft2, ifft2
import matplotlib.pyplot as plt
import scipy

In [19]:
def wiener_filter(img, kernel, K):
    dummy = np.copy(img)
    kernel = np.pad(kernel, [(0, dummy.shape[0] - kernel.shape[0]), (0, dummy.shape[1] - kernel.shape[1])], 'constant')
    # Fourier Transform
    dummy = fft2(dummy)
    kernel = fft2(kernel)
    kernel = np.conj(kernel) / (np.abs(kernel) ** 2 + K)
    dummy = dummy * kernel
    dummy = np.abs(ifft2(dummy))
    return np.uint8(dummy)

In [20]:
def connectedComponents(thresh1):
    ## Remove small object in vessel_01
            #find all your connected components (white blobs in your image)
    bin_uint8 = (thresh1 * 255).astype(np.uint8)
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(bin_uint8, connectivity=8)
            #connectedComponentswithStats yields every seperated component with information on each of them, such as size
            #the following part is just taking out the background which is also considered a component, but most of the time we don't want that.
    sizes = stats[1:, -1]; nb_components = nb_components - 1
            # minimum size of particles we want to keep (number of pixels)
            #here, it's a fixed value, but you can set it as you want, eg the mean of the sizes or whatever
    min_size = 100
            ## Vessel connected component
    vessel_01 = np.zeros((output.shape))
            #for every component in the image, you keep it only if it's above min_size
    for ii in range(0, nb_components):
        if sizes[ii] >= min_size:
            vessel_01[output == ii + 1] = 255
    return vessel_01


In [21]:
def masking(vessel_01,mask):
    ret,thresh1 = cv.threshold(vessel_01,150,255,cv.THRESH_BINARY)
    thresh1=thresh1/255
    ret1,thresh_mask = cv.threshold(mask,120,255,cv.THRESH_BINARY)
    thresh_mask=thresh_mask/255
    new=thresh_mask*th3
    se = cv.getStructuringElement(cv.MORPH_ELLIPSE,(9,9))
    maskIdx = cv.erode(thresh_mask,se,iterations = 2)
    newImg = thresh_mask* np.uint8(maskIdx);
    without_border=newImg*thresh1;
    cv.imshow('img_without_border',without_border)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return without_border

In [22]:
def tp_tn(img_ground_truth,wid_border):
    idx_tp = 0
    idx_tn = 0
    idx_fp = 0
    idx_fn = 0
    img_tp=np.zeros_like(wid_border)
    img_tn=np.zeros_like(wid_border)
    img_fp=np.zeros_like(wid_border)
    img_fn=np.zeros_like(wid_border)

    for i in range(wid_border.shape[0]):
        for j in range(wid_border.shape[1]):
            if wid_border[i,j]==img_ground_truth[i,j] and wid_border[i,j]==1.0:
                idx_tp +=1
                img_tp[i,j]=255
            elif wid_border[i,j]==img_ground_truth[i,j] and wid_border[i,j]==0.0:
                idx_tn +=1
                img_tn[i,j]=255
            elif wid_border[i,j]!=img_ground_truth[i,j] and wid_border[i,j]==1.0:
                idx_fp +=1
                img_fp[i,j]=255
            elif wid_border[i,j]!=img_ground_truth[i,j] and wid_border[i,j]==0.0:
                idx_fn +=1
                img_fn[i,j]=255
    cv.imshow('img_tp',img_tp)
    cv.imshow('img_tn',img_tn)
    cv.imshow('img_fp',img_fp)
    cv.imshow('img_fn',img_fn)
    cv.waitKey(0)
    cv.destroyAllWindows()

    print("True positives are",idx_tp)
    print("True negatives are",idx_tn)
    print("False positives are",idx_fp)
    print("False negatives are",idx_fn)
    print("Sensitivity is",idx_tp/(idx_tp+idx_fn))
    print("Specificity is",idx_tn/(idx_tn+idx_fp))
    print("Accuracy",(idx_tp+idx_tn)/(idx_tp+idx_tn+idx_fp+idx_fn))


In [23]:
img = cv.imread('2_training.jpg')
mask=cv.imread('02_mask.jpg',cv.IMREAD_GRAYSCALE)

b,g,r=cv.split(img)

clahe = cv.createCLAHE(clipLimit=3, tileGridSize=(8,8))
enhanced_image = clahe.apply(g)

th = cv.adaptiveThreshold(enhanced_image,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C ,cv.THRESH_BINARY,15,2);

not_th=cv.bitwise_not(th)

not_converted_rgb= cv.cvtColor(not_th, cv.COLOR_GRAY2RGB)

img_Lab= cv.cvtColor(not_converted_rgb, cv.COLOR_RGB2Lab)

kernel = np.array([[-1,-1,-1],
                   [-1, 10,-1],
                   [-1,-1,-1]])

L,a,b=cv.split(img_Lab)

sharpened = cv.filter2D(L, -1, kernel)

img_Lab[:,:,1]=sharpened
final_sharpened= cv.cvtColor(img_Lab, cv.COLOR_RGB2GRAY)

# Perform Weiner Filtering
w_kernel = np.array([[0.0128,0.0876,0.0128],
                   [0.0876,0.5986,0.0876],
                   [0.0128,0.0876,0.0128]])

Weiner = wiener_filter(final_sharpened, w_kernel, 50)

# Perform Otsu Thresholding
ret3,th3 = cv.threshold(Weiner,0,255,cv.THRESH_OTSU)

# Perform Morphological Opening to extract the blood vessels.
m_kernel = np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]]
                   ,np.uint8)
opening = cv.morphologyEx(th3, cv.MORPH_OPEN, m_kernel)

opening_dilate = cv.dilate(opening,m_kernel,iterations = 1)

ret,thresh1 = cv.threshold(opening_dilate,150,255,cv.THRESH_BINARY)
thresh1=thresh1/255

img_ground_truth=cv.imread('02_manual1.jpg',cv.IMREAD_GRAYSCALE)
img_ground_truth=img_ground_truth/255

connected_comp_img = connectedComponents(thresh1)

masked_img = masking(connected_comp_img,mask)

tp_tn(img_ground_truth, masked_img)


True positives are 16359
True negatives are 248379
False positives are 33677
False negatives are 31545
Sensitivity is 0.3414954909819639
Specificity is 0.8806017244802451
Accuracy 0.8023336161959025
